In [1]:
%pip install biopython
import Bio

import os

# List files in the current directory
print(os.listdir())

Note: you may need to restart the kernel to use updated packages.
[' final_test_dataset.csv', ' final_train_dataset.csv', 'Dataset', 'Dataset_output', 'help.ipynb', 'human_IDPPI.fasta', 'human_ParkMarcotte.fasta', 'new_for_next.ipynb', 'ngram_feature_vector.csv', 'ngram_feature_vector_new.csv', 'pair_formatted_dataset.tsv', 'Project_all_part.ipynb', 'TestSet1.csv', 'TestSet2.csv', 'TestSet3.csv', 'test_dataset11.tsv', 'TrainSet1.csv', 'TrainSet2.csv', 'TrainSet3.csv', 'train_dataset11.tsv']



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\cs21b\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
file_path = r"C:\Users\cs21b\Desktop\Major_project_trail\human_IDPPI.fasta"

In [4]:
file_path = r"C:\Users\cs21b\Desktop\Major_project_trail\human_ParkMarcotte.fasta"

protein_data = {}  # Dictionary to store ID: sequence

with open(file_path, "r") as f:
    current_id = None
    current_seq = []

    for line in f:
        line = line.strip()
        if line.startswith(">"):
            if current_id:  # Save the previous entry
                protein_data[current_id] = "".join(current_seq)
            # Extract protein ID (e.g., P27540)
            parts = line.split("|")
            if len(parts) > 1:
                current_id = parts[1]
            else:
                current_id = line[1:].split()[0]  # Fallback if '|' not found
            current_seq = []
        else:
            current_seq.append(line)

    # Save last sequence
    if current_id:
        protein_data[current_id] = "".join(current_seq)

print("Total protein entries:", len(protein_data))
print("\n\n\n")


# Optional: Check first few entries
for pid, seq in list(protein_data.items())[:3]:
    print(f"{pid} : {seq[:50]}... (length={len(seq)})")


Total protein entries: 20117




P00441 : MATKAVCVLKGDGPVQGIINFEQKESNGPVKVWGSIKGLTEGLHGFHVHE... (length=154)
Q13046 : MGPLSAPPCTQHITWKGLLLTASLLNFWNPPTTAQVTIEAQPPKVSEGKD... (length=419)
P0C868 : MVVDLKNLLFNPSKPVSRGSQPADVDLMIDCLVSCLRVSPHNNQQFKIFL... (length=119)


In [5]:
# import numpy as np
# from itertools import product

# # Step 1: Define all possible 2-grams
# amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
# two_grams = [''.join(p) for p in product(amino_acids, repeat=2)]
# gram_index = {gram: idx for idx, gram in enumerate(two_grams)}  # Map: dipeptide -> index

# # Step 2: Convert each sequence into 400-dim vector
# def sequence_to_2gram_vector(sequence):
#     vector = np.zeros(400, dtype=np.float32)
#     for i in range(len(sequence) - 1):
#         dipeptide = sequence[i:i+2]
#         if dipeptide in gram_index:
#             vector[gram_index[dipeptide]] += 1
#     return vector

# # Step 3: Apply to all sequences
# protein_vectors = {}
# for pid, seq in protein_data.items():
#     protein_vectors[pid] = sequence_to_2gram_vector(seq)

# # Optional: check shape and sample
# sample_id = list(protein_vectors.keys())[0]
# print("Sample ID:", sample_id)
# print("Vector shape:", protein_vectors[sample_id].shape)
# print("Vector (first 10):", protein_vectors[sample_id][:10])

# === Step 2: 2-gram model setup ===
# amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
# two_grams = [''.join(p) for p in product(amino_acids, repeat=2)]
# gram_index = {gram: idx for idx, gram in enumerate(two_grams)}

# def sequence_to_2gram_vector(sequence):
#     vector = np.zeros(400, dtype=np.float32)
#     for i in range(len(sequence) - 1):
#         dipeptide = sequence[i:i+2]
#         if dipeptide in gram_index:
#             vector[gram_index[dipeptide]] += 1
#     return vector

# # === Step 3: Generate vectors for all protein sequences ===
# protein_vectors = {}

# for pid, seq in protein_data.items():
#     protein_vectors[pid] = sequence_to_2gram_vector(seq)

# # Optional check
# # sample_id = list(protein_vectors.keys())[0]
# # print("Sample ID:", sample_id)
# # print("Vector shape:", protein_vectors[sample_id].shape)
# # print("Vector (first 10 values):", protein_vectors[sample_id][:10])

# # Print the full vector for a specific protein ID
# sample_id = list(protein_vectors.keys())[0]  # First protein ID in the dictionary
# print(f"Full vector for Protein ID {sample_id}:")
# print(protein_vectors[sample_id])
# # import numpy as np

# # Convert all vectors to a single NumPy array
# all_vectors = np.array(list(protein_vectors.values()))

# print("Shape of feature matrix:", all_vectors.shape)  # (total_sequences, 400)


import numpy as np
import pandas as pd
from Bio import SeqIO

# Amino acids (standard 20)
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
aa_index = {aa: i for i, aa in enumerate(amino_acids)}

# Function to create 20x20 bigram frequency matrix
def create_bigram_matrix(sequence):
    matrix = np.zeros((20, 20), dtype=float)
    total_bigrams = 0

    for i in range(len(sequence) - 1):
        a1, a2 = sequence[i], sequence[i + 1]
        if a1 in aa_index and a2 in aa_index:
            matrix[aa_index[a1], aa_index[a2]] += 1
            total_bigrams += 1

    if total_bigrams > 0:
        matrix /= total_bigrams  # Normalize

    return matrix.flatten()  # Convert to 400-dim feature vector

# Path to the FASTA file
file_path = r"C:\Users\cs21b\Desktop\Major_project_trail\human_ParkMarcotte.fasta"

# Process all sequences and generate feature vectors
feature_vectors = []
seq_ids = []

for record in SeqIO.parse(file_path, "fasta"):
    seq = str(record.seq)
    vec = create_bigram_matrix(seq)
    feature_vectors.append(vec)
    seq_ids.append(record.id)

# Create DataFrame with 400-dimensional feature vectors
columns = [f'{a1}{a2}' for a1 in amino_acids for a2 in amino_acids]
df = pd.DataFrame(feature_vectors, columns=columns)
df.insert(0, "Sequence_ID", seq_ids)

# Save to CSV
output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
df.to_csv(output_path, index=False)

# Optional: Check DataFrame shape and first few rows
print("DataFrame shape:", df.shape)
print(df.head())



DataFrame shape: (20117, 401)
  Sequence_ID        AA        AC        AD        AE        AF        AG  \
0      P00441  0.000000  0.006536  0.019608  0.000000  0.000000  0.019608   
1      Q13046  0.000000  0.002392  0.002392  0.000000  0.000000  0.002392   
2      P0C868  0.000000  0.000000  0.008475  0.000000  0.000000  0.000000   
3      Q96G97  0.002519  0.000000  0.002519  0.002519  0.002519  0.005038   
4      Q9UBN7  0.014003  0.004119  0.003295  0.007414  0.000824  0.008237   

         AH        AI        AK  ...        YM        YN        YP        YQ  \
0  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.002392  0.002392   
2  0.000000  0.000000  0.008475  ...  0.000000  0.000000  0.000000  0.000000   
3  0.005038  0.002519  0.000000  ...  0.005038  0.002519  0.000000  0.000000   
4  0.001647  0.004119  0.000824  ...  0.000824  0.001647  0.000000  0.000824   

         YR        YS     

In [5]:
# import pandas as pd
# import numpy as np
# from itertools import combinations

# # Read the CSV
# csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# df = pd.read_csv(csv_path)

# # Separate IDs and vectors
# ids = df['Sequence_ID'].values  # Array of protein IDs
# X = df.drop(columns=['Sequence_ID']).values  # Feature vectors

# # Prepare lines
# lines = []
# pairs = list(combinations(range(len(ids)), 2))  # All unique pairs

# for i, j in pairs:
#     id1 = ids[i]
#     id2 = ids[j]
#     pair_id = f"{id1}+{id2}"
#     vec_i = X[i]
#     vec_j = X[j]
#     combined_features = np.concatenate([vec_i, vec_j])
#     features_str = ' '.join(map(str, combined_features))  # space-separated features
#     label = ''  # Placeholder for label
#     line = f"{pair_id}\t{features_str}\t{label}"  # tab-separated line
#     lines.append(line)

# # Save to TSV
# output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\pair_formatted_dataset_new.tsv"
# with open(output_path, 'w') as f:
#     for line in lines:
#         f.write(line + '\n')


In [ ]:
# # # # # import pandas as pd
# # # # # import numpy as np

# # # # # # Load precomputed 400-dim vectors from CSV
# # # # # vector_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# # # # # vector_df = pd.read_csv(vector_csv_path)

# # # # # # Make a dictionary: ID → vector
# # # # # vector_dict = {}
# # # # # for _, row in vector_df.iterrows():
# # # # #     vector_dict[row['Sequence_ID']] = row.drop('Sequence_ID').values.astype(float)

# # # # # # Function to generate paired dataset
# # # # # def generate_pair_dataset(pair_csv_path, output_path):
# # # # #     pair_df = pd.read_csv(pair_csv_path)
# # # # #     lines = []

# # # # #     for _, row in pair_df.iterrows():
# # # # #         id1, id2, label = row['IntA'], row['IntB'], row['Cls']

# # # # #         if id1 in vector_dict and id2 in vector_dict:
# # # # #             vec1 = vector_dict[id1]
# # # # #             vec2 = vector_dict[id2]
# # # # #             combined = np.concatenate([vec1, vec2])
# # # # #             combined_str = ' '.join(map(str, combined))
# # # # #             line = f"{id1}+{id2}\t{combined_str}\t{label}"
# # # # #             lines.append(line)
# # # # #         else:
# # # # #             print(f"Missing vector for: {id1} or {id2}")

# # # # #     # Save to file
# # # # #     with open(output_path, 'w') as f:
# # # # #         for line in lines:
# # # # #             f.write(line + '\n')

# # # # # # Paths
# # # # # train_csv = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet1.csv"
# # # # # test_csv = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet1.csv"
# # # # # train_output = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet111.csv"
# # # # # test_output = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet111.csv"

# # # # # # Generate datasets
# # # # # generate_pair_dataset(train_csv, train_output)
# # # # # generate_pair_dataset(test_csv, test_output)


# # # # # import pandas as pd
# # # # # import numpy as np

# # # # # # Load 400-dim vectors
# # # # # vector_df = pd.read_csv(r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv")
# # # # # vector_dict = {
# # # # #     row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
# # # # #     for _, row in vector_df.iterrows()
# # # # # }

# # # # # # Function to generate dataset
# # # # # def create_dataset(pair_file, output_file):
# # # # #     pairs_df = pd.read_csv(pair_file)
# # # # #     lines = []

# # # # #     for _, row in pairs_df.iterrows():
# # # # #         id1, id2, label = row['IntA'], row['IntB'], row['Cls']

# # # # #         if id1 in vector_dict and id2 in vector_dict:
# # # # #             vec1 = vector_dict[id1]
# # # # #             vec2 = vector_dict[id2]

# # # # #             vec1_str = ' '.join(map(str, vec1))
# # # # #             vec2_str = ' '.join(map(str, vec2))

# # # # #             line = f"{id1}+{id2}\t{vec1_str}\t{vec2_str}\t{label}"
# # # # #             lines.append(line)
# # # # #         else:
# # # # #             print(f" Missing vector for: {id1} or {id2}")

# # # # #     # Write to output
# # # # #     with open(output_file, 'w') as f:
# # # # #         for line in lines:
# # # # #             f.write(line + '\n')

# # # # # # Paths
# # # # # train_input = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet1.csv"
# # # # # test_input = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet1.csv"
# # # # # train_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Train_Output1.tsv"
# # # # # test_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Test_Output1.tsv"

# # # # # # Generate both datasets
# # # # # create_dataset(train_input, train_output)
# # # # # create_dataset(test_input, test_output)


# # # # import pandas as pd
# # # # import numpy as np

# # # # # --- Load feature vectors from CSV ---
# # # # feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# # # # fv_df = pd.read_csv(feature_vector_path)

# # # # # Convert to dict: ID -> 400-dim vector
# # # # feature_vectors = {}
# # # # for _, row in fv_df.iterrows():
# # # #     pid = row['Sequence_ID']
# # # #     vector = row.drop('Sequence_ID').values.astype(float)
# # # #     feature_vectors[pid] = vector

# # # # # --- ID Cleaning Function ---
# # # # def clean_id(protein_id):
# # # #     # Extract ID like P27540 from sp|P27540|ARNT_HUMAN
# # # #     if "|" in protein_id:
# # # #         parts = protein_id.split("|")
# # # #         if len(parts) > 1:
# # # #             return parts[1]
# # # #     return protein_id

# # # # # --- Function to create final dataset ---
# # # # def create_pair_dataset(input_csv_path, output_tsv_path):
# # # #     df = pd.read_csv(input_csv_path)

# # # #     lines = []
# # # #     missing_count = 0

# # # #     for _, row in df.iterrows():
# # # #         id1 = clean_id(row['IntA'])
# # # #         id2 = clean_id(row['IntB'])
# # # #         label = row['Cls']

# # # #         if id1 in feature_vectors and id2 in feature_vectors:
# # # #             vec1 = feature_vectors[id1]
# # # #             vec2 = feature_vectors[id2]
# # # #             pair_id = f"{id1}+{id2}"
# # # #             combined_vec = np.concatenate([vec1, vec2])
# # # #             combined_str = " ".join(map(str, combined_vec))
# # # #             line = f"{pair_id}\t{combined_str}\t{label}"
# # # #             lines.append(line)
# # # #         else:
# # # #             missing_count += 1

# # # #     print(f"Total valid pairs: {len(lines)}")
# # # #     print(f"Missing pairs (not found in feature vectors): {missing_count}")

# # # #     # Write to TSV
# # # #     with open(output_tsv_path, 'w') as f:
# # # #         for line in lines:
# # # #             f.write(line + '\n')

# # # # # --- File paths ---
# # # # train_input = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet1.csv"
# # # # train_output = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"

# # # # test_input = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet1.csv"
# # # # test_output = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"

# # # # # --- Run ---
# # # # create_pair_dataset(train_input, train_output)
# # # # create_pair_dataset(test_input, test_output)


# # # import pandas as pd
# # # import numpy as np
# # # import os

# # # # --- Load feature vectors ---
# # # feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# # # fv_df = pd.read_csv(feature_vector_path)
# # # feature_vectors = {
# # #     row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
# # #     for _, row in fv_df.iterrows()
# # # }

# # # # --- ID Cleaning Function ---
# # # def clean_id(protein_id):
# # #     if "|" in protein_id:
# # #         parts = protein_id.split("|")
# # #         if len(parts) > 1:
# # #             return parts[1]
# # #     return protein_id

# # # # --- Function to create dataset ---
# # # def create_pair_dataset(input_csv_path, output_tsv_path):
# # #     df = pd.read_csv(input_csv_path)

# # #     lines = []
# # #     missing_count = 0

# # #     for _, row in df.iterrows():
# # #         id1 = clean_id(row['IntA'])
# # #         id2 = clean_id(row['IntB'])
# # #         label = row['Cls']

# # #         if id1 in feature_vectors and id2 in feature_vectors:
# # #             vec1 = feature_vectors[id1]
# # #             vec2 = feature_vectors[id2]
# # #             pair_id = f"{id1}+{id2}"
# # #             combined_vec = np.concatenate([vec1, vec2])
# # #             combined_str = " ".join(map(str, combined_vec))
# # #             line = f"{pair_id}\t{combined_str}\t{label}"
# # #             lines.append(line)
# # #         else:
# # #             missing_count += 1

# # #     print(f"{os.path.basename(input_csv_path)} -> Valid: {len(lines)}, Missing: {missing_count}")

# # #     # Write output
# # #     os.makedirs(os.path.dirname(output_tsv_path), exist_ok=True)
# # #     with open(output_tsv_path, 'w') as f:
# # #         for line in lines:
# # #             f.write(line + '\n')

# # # # --- Dataset paths ---
# # # datasets = [
# # #   {  train_input11 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet1.csv"
# # #     train_output11 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset11.tsv"
# # #     test_input11 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet1.csv"
# # #     test_output11 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset11.tsv"
# # # },
# # #   {  train_input12 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet2.csv"
# # #     train_output12 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset12.tsv"
# # #     test_input12 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet2.csv"
# # #     test_output12 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset12.tsv"
# # # },
# # #   {  train_input13 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet3.csv"
# # #     train_output13 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset13.tsv"
# # #     test_input13 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet3.csv"
# # #     test_output13 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset13.tsv"
# # # },
# # #   {  train_input14 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet4.csv"
# # #     train_output14 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset14.tsv"
# # #     test_input14 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet4.csv"
# # #     test_output14 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset14.tsv"
# # # },
# # #   {  train_input15 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet5.csv"
# # #     train_output15 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset15.tsv"
# # #     test_input15 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet5.csv"
# # #     test_output15 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset15.tsv"
# # # },
# # #   {  train_input21 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TrainSet1.csv"
# # #     train_output21 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Train\train_dataset21.tsv"
# # #     test_input21 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TestSet1.csv"
# # #     test_output21 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Test\test_dataset21.tsv"
# # # },
# # #   {  train_input22 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TrainSet2.csv"
# # #     train_output22 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Train\train_dataset22.tsv"
# # #     test_input22 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TestSet2.csv"
# # #     test_output22 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Test\test_dataset22.tsv"
# # # },
# # #   {  train_input23 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TrainSet3.csv"
# # #     train_output23 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Train\train_dataset23.tsv"
# # #     test_input23 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TestSet3.csv"
# # #     test_output23 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Test\test_dataset23.tsv"
# # # },
# # #   {  train_input24 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TrainSet4.csv"
# # #     train_output24 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Train\train_dataset24.tsv"
# # #     test_input24 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TestSet4.csv"
# # #     test_output24 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Test\test_dataset24.tsv"
# # # },
# # #   {  train_input25 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TrainSet5.csv"
# # #     train_output25 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Train\train_dataset25.tsv"
# # #     test_input25 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C2\TestSet5.csv"
# # #     test_output25 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C2\For_Test\test_dataset25.tsv"
# # # },
# # #   {  train_input31 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TrainSet1.csv"
# # #     train_output31 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Train\train_dataset31.tsv"
# # #     test_input31 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TestSet1.csv"
# # #     test_output31 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Test\test_dataset31.tsv"
# # # },
# # #   {  train_input32 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TrainSet2.csv"
# # #     train_output32 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Train\train_dataset32.tsv"
# # #     test_input32 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TestSet2.csv"
# # #     test_output32 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Test\test_dataset32.tsv"
# # # },
# # #   {  train_input33 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TrainSet3.csv"
# # #     train_output33 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Train\train_dataset33.tsv"
# # #     test_input33 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TestSet3.csv"
# # #     test_output33 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Test\test_dataset33.tsv"
# # # },
# # #  {   train_input34 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TrainSet4.csv"
# # #     train_output34 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Train\train_dataset34.tsv"
# # #     test_input34 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TestSet4.csv"
# # #     test_output34 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Test\test_dataset34.tsv"
# # # },
# # #     {train_input35 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TrainSet5.csv"
# # #     train_output35 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Train\train_dataset35.tsv"
# # #     test_input35 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C3\TestSet5.csv"
# # #     test_output35 = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C3\For_Test\test_dataset35.tsv"
# # # },

# # # ]

# # # # --- Run for all datasets ---
# # # for dataset in datasets:
# # #     create_pair_dataset(dataset["train_input"], dataset["train_output"])
# # #     create_pair_dataset(dataset["test_input"], dataset["test_output"])



# # import pandas as pd
# # import numpy as np

# # # --- Load feature vectors from CSV ---
# # feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# # fv_df = pd.read_csv(feature_vector_path)

# # # Convert to dict: ID -> 400-dim vector
# # feature_vectors = {
# #     row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
# #     for _, row in fv_df.iterrows()
# # }

# # # --- ID Cleaning Function ---
# # def clean_id(protein_id):
# #     if "|" in protein_id:
# #         parts = protein_id.split("|")
# #         if len(parts) > 1:
# #             return parts[1]
# #     return protein_id

# # # --- Function to create final dataset ---
# # def create_pair_dataset(input_csv_path, output_tsv_path):
# #     df = pd.read_csv(input_csv_path)
# #     lines = []
# #     missing_count = 0

# #     for _, row in df.iterrows():
# #         id1 = clean_id(row['IntA'])
# #         id2 = clean_id(row['IntB'])
# #         label = row['Cls']

# #         if id1 in feature_vectors and id2 in feature_vectors:
# #             vec1 = feature_vectors[id1]
# #             vec2 = feature_vectors[id2]
# #             pair_id = f"{id1}+{id2}"
# #             combined_vec = np.concatenate([vec1, vec2])
# #             combined_str = " ".join(map(str, combined_vec))
# #             line = f"{pair_id}\t{combined_str}\t{label}"
# #             lines.append(line)
# #         else:
# #             missing_count += 1

# #     # print(f" {input_csv_path} → {output_tsv_path}")
# #     print(f"   Total valid pairs: {len(lines)}")
# #     print(f"   Missing pairs: {missing_count}\n\n")

# #     with open(output_tsv_path, 'w') as f:
# #         for line in lines:
# #             f.write(line + '\n')

# # # --- List of input-output file path pairs ---
# # file_paths = [
# #     (
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet1.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset11.tsv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet1.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset11.tsv",
# #     ),
# #     (
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet2.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset12.tsv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet2.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset12.tsv",
# #     ),
# #     (
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet3.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset13.tsv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet3.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset13.tsv",
# #     ),
# #     (
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet4.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset14.tsv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet4.csv",
# #         r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset14.tsv",
# #     ),
# # ]

# # # --- Run loop for each pair ---
# # for train_in, train_out, test_in, test_out in file_paths:
# #     create_pair_dataset(train_in, train_out)
# #     create_pair_dataset(test_in, test_out)


# import pandas as pd
# import numpy as np
# import os

# # --- Load feature vectors ---
# feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# fv_df = pd.read_csv(feature_vector_path)
# feature_vectors = {
#     row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
#     for _, row in fv_df.iterrows()
# }

# # --- ID Cleaning Function ---
# def clean_id(protein_id):
#     if "|" in protein_id:
#         parts = protein_id.split("|")
#         if len(parts) > 1:
#             return parts[1]
#     return protein_id

# # --- Function to create dataset ---
# def create_pair_dataset(input_csv_path, output_tsv_path):
#     df = pd.read_csv(input_csv_path)

#     lines = []
#     missing_count = 0

#     for _, row in df.iterrows():
#         id1 = clean_id(row['IntA'])
#         id2 = clean_id(row['IntB'])
#         label = row['Cls']

#         if id1 in feature_vectors and id2 in feature_vectors:
#             vec1 = feature_vectors[id1]
#             vec2 = feature_vectors[id2]
#             pair_id = f"{id1}+{id2}"
#             combined_vec = np.concatenate([vec1, vec2])
#             combined_str = " ".join(map(str, combined_vec))
#             line = f"{pair_id}\t{combined_str}\t{label}"
#             lines.append(line)
#         else:
#             missing_count += 1

#     print(f"{os.path.basename(input_csv_path)} -> Valid: {len(lines)}, Missing: {missing_count}")

#     # Write output
#     os.makedirs(os.path.dirname(output_tsv_path), exist_ok=True)
#     with open(output_tsv_path, 'w') as f:
#         for line in lines:
#             f.write(line + '\n')

# # --- Dataset paths ---
# datasets = [
#     {
#         "train_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet1.csv",
#         "train_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset11.tsv",
#         "test_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet1.csv",
#         "test_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset11.tsv",
#     },
#     {
#         "train_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet2.csv",
#         "train_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset12.tsv",
#         "test_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet2.csv",
#         "test_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset12.tsv",
#     },
#     {
#         "train_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet3.csv",
#         "train_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset13.tsv",
#         "test_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet3.csv",
#         "test_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset13.tsv",
#     },
#     {
#         "train_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TrainSet4.csv",
#         "train_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Train\train_dataset14.tsv",
#         "test_input": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset\Dataset_C1\TestSet4.csv",
#         "test_output": r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output\For_C1\For_Test\test_dataset14.tsv",
#     },
# ]

# # --- Run for all datasets ---
# for dataset in datasets:
#     create_pair_dataset(dataset["train_input"], dataset["train_output"])
#     create_pair_dataset(dataset["test_input"], dataset["test_output"])


# import pandas as pd
# import numpy as np
# import os

# # --- Load feature vectors from CSV ---
# feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
# fv_df = pd.read_csv(feature_vector_path)
# feature_vectors = {
#     row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
#     for _, row in fv_df.iterrows()
# }

# # --- Clean protein ID ---
# def clean_id(protein_id):
#     if "|" in protein_id:
#         parts = protein_id.split("|")
#         if len(parts) > 1:
#             return parts[1]
#     return protein_id

# # --- Function to create final dataset ---
# def create_pair_dataset(input_csv_path, output_tsv_path):
#     df = pd.read_csv(input_csv_path)
#     lines = []
#     missing_count = 0

#     for _, row in df.iterrows():
#         id1 = clean_id(row['IntA'])
#         id2 = clean_id(row['IntB'])
#         label = row['Cls']

#         if id1 in feature_vectors and id2 in feature_vectors:
#             vec1 = feature_vectors[id1]
#             vec2 = feature_vectors[id2]
#             pair_id = f"{id1}+{id2}"
#             combined_vec = np.concatenate([vec1, vec2])
#             combined_str = " ".join(map(str, combined_vec))
#             line = f"{pair_id}\t{combined_str}\t{label}"
#             lines.append(line)
#         else:
#             missing_count += 1

#     print(f"{os.path.basename(input_csv_path)} → Valid pairs: {len(lines)}, Missing: {missing_count}")

#     with open(output_tsv_path, 'w') as f:
#         for line in lines:
#             f.write(line + '\n')

# # --- All input/output pairs ---
# dataset_paths = [
#     # C1
#     (11, "C1"), (12, "C1"), (13, "C1"), (14, "C1"), (15, "C1"),
#     # C2
#     (21, "C2"), (22, "C2"), (23, "C2"), (24, "C2"), (25, "C2"),
#     # C3
#     (31, "C3"), (32, "C3"), (33, "C3"), (34, "C3"), (35, "C3"),
# ]

# # --- Run in loop ---
# base_input = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset"
# base_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# for num, cat in dataset_paths:
#     train_input = fr"{base_input}\Dataset_{cat}\TrainSet{str(num)[-1]}.csv"
#     train_output = fr"{base_output}\For_{cat}\For_Train\train_dataset{num}.tsv"
#     test_input = fr"{base_input}\Dataset_{cat}\TestSet{str(num)[-1]}.csv"
#     test_output = fr"{base_output}\For_{cat}\For_Test\test_dataset{num}.tsv"

#     create_pair_dataset(train_input, train_output)
#     create_pair_dataset(test_input, test_output)


import pandas as pd
import numpy as np
import os

# --- Load feature vectors from CSV ---
feature_vector_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
fv_df = pd.read_csv(feature_vector_path)
feature_vectors = {
    row['Sequence_ID']: row.drop('Sequence_ID').values.astype(float)
    for _, row in fv_df.iterrows()
}

# --- Clean protein ID ---
def clean_id(protein_id):
    if "|" in protein_id:
        parts = protein_id.split("|")
        if len(parts) > 1:
            return parts[1]
    return protein_id

# --- Function to create final dataset ---
def create_pair_dataset(input_csv_path, output_tsv_path):
    df = pd.read_csv(input_csv_path)

    # Check for expected columns
    required_cols = {'IntA', 'IntB', 'Cls'}
    if not required_cols.issubset(df.columns): 
        # print(f"[!] Missing columns in {os.path.basename(input_csv_path)} → Found: {df.columns.tolist()}")
        required_cols = {'IntA', 'IntB', 'Class'}
        if not required_cols.issubset(df.columns): 
           print(f"[!] Missing columns in {os.path.basename(input_csv_path)} → Found: 'IntA', 'IntB', 'Cls' or 'Class'")
           return

    lines = []
    missing_count = 0

    for _, row in df.iterrows():
        id1 = clean_id(row['IntA'])
        id2 = clean_id(row['IntB'])
        # label = row['Cls']
        
        if 'Cls' in row:
            label = row['Cls']
        elif 'Class' in row:
            label = row['Class']
        else:
            # print(" Label column not found in row")
            continue


        if id1 in feature_vectors and id2 in feature_vectors:
            vec1 = feature_vectors[id1]
            vec2 = feature_vectors[id2]
            pair_id = f"{id1}+{id2}"
            combined_vec = np.concatenate([vec1, vec2])
            combined_str = " ".join(map(str, combined_vec))
            line = f"{pair_id}\t{combined_str}\t{label}"
            lines.append(line)
        else:
            missing_count += 1

    print(f"{os.path.basename(input_csv_path)} → Valid pairs: {len(lines)}, Missing: {missing_count}")

    os.makedirs(os.path.dirname(output_tsv_path), exist_ok=True)
    with open(output_tsv_path, 'w') as f:
        for line in lines:
            f.write(line + '\n')

# --- All input/output pairs ---
dataset_paths = {
    "C1": [11, 12, 13, 14, 15],
    "C2": [21, 22, 23, 24, 25],
    "C3": [31, 32, 33, 34, 35],
}

# --- Run in loop ---
base_input = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset"
base_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

for category, nums in dataset_paths.items():
    print(f"\n For DataSet inside {category} :- ")
    for num in nums:
        idx = str(num)[-1]  # Get 1-5 for C1, etc.

        train_input = fr"{base_input}\Dataset_{category}\TrainSet{idx}.csv"
        train_output = fr"{base_output}\For_{category}\For_Train\train_dataset{num}.tsv"
        test_input = fr"{base_input}\Dataset_{category}\TestSet{idx}.csv"
        test_output = fr"{base_output}\For_{category}\For_Test\test_dataset{num}.tsv"

        create_pair_dataset(train_input, train_output)
        create_pair_dataset(test_input, test_output)




 For DataSet inside C1 :- 
TrainSet1.csv → Valid pairs: 45380, Missing: 124
TestSet1.csv → Valid pairs: 5049, Missing: 7
TrainSet2.csv → Valid pairs: 45383, Missing: 121
TestSet2.csv → Valid pairs: 5046, Missing: 10
TrainSet3.csv → Valid pairs: 45380, Missing: 124
TestSet3.csv → Valid pairs: 5049, Missing: 7
TrainSet4.csv → Valid pairs: 45386, Missing: 118
TestSet4.csv → Valid pairs: 5043, Missing: 13
TrainSet5.csv → Valid pairs: 45384, Missing: 120
TestSet5.csv → Valid pairs: 5045, Missing: 11

 For DataSet inside C2 :- 
TrainSet1.csv → Valid pairs: 18940, Missing: 122
TestSet1.csv → Valid pairs: 1496, Missing: 4
TrainSet2.csv → Valid pairs: 18356, Missing: 92
TestSet2.csv → Valid pairs: 1497, Missing: 3
TrainSet3.csv → Valid pairs: 18982, Missing: 57
TestSet3.csv → Valid pairs: 1488, Missing: 12
TrainSet4.csv → Valid pairs: 19237, Missing: 86
TestSet4.csv → Valid pairs: 1491, Missing: 9
TrainSet5.csv → Valid pairs: 19176, Missing: 92
TestSet5.csv → Valid pairs: 1499, Missing: 1

 Fo

In [ ]:
# import pandas as pd
# import numpy as np

# # === Step 1: Load feature vectors ===
# features_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"  # Dummy path

# # Load and clean IDs like sp|P27540|ARNT_HUMAN --> P27540
# features_df = pd.read_csv(features_path, header=None)
# features_df.rename(columns={0: "Raw_ID"}, inplace=True)
# features_df["Sequence_ID"] = features_df["Raw_ID"].apply(lambda x: x.split('|')[1] if '|' in x else x)
# features_df.drop(columns=["Raw_ID"], inplace=True)

# # Create feature dictionary
# feature_dict = {
#     row["Sequence_ID"]: row[1:].values.astype(float)
#     for _, row in features_df.iterrows()
# }

# # === Step 2: Load interaction pairs ===
# train_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet1.csv"  # Dummy path
# test_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet1.csv"    # Dummy path

# train_df = pd.read_csv(train_csv_path)
# test_df = pd.read_csv(test_csv_path)

# # === Step 3: Function to generate formatted data ===
# def generate_formatted_lines(pairs_df, feature_dict):
#     lines = []
    
#     for _, row in pairs_df.iterrows():
#         id1 = str(row["IntA"])
#         id2 = str(row["IntB"])
#         label = str(row["Cls"])

#         if id1 in feature_dict and id2 in feature_dict:
#             vec1 = feature_dict[id1]
#             vec2 = feature_dict[id2]
#             combined = np.concatenate([vec1, vec2])
#             vec_str = ' '.join(map(str, combined))
#             line = f"{id1}+{id2}\t{vec_str}\t{label}"
#             lines.append(line)
#         else:
#             print(f"Skipping pair (missing vector): {id1}, {id2}")
    
#     return lines

# # === Step 4: Generate and write outputs ===
# train_lines = generate_formatted_lines(train_df, feature_dict)
# test_lines = generate_formatted_lines(test_df, feature_dict)

# train_output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"  # Dummy path
# test_output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"    # Dummy path

# with open(train_output_path, 'w') as f:
#     for line in train_lines:
#         f.write(line + '\n')

# with open(test_output_path, 'w') as f:
#     for line in test_lines:
#         f.write(line + '\n')

# print(" Train and test datasets saved successfully!")


In [ ]:
# import pandas as pd
# import numpy as np


# features_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"

# # Load CSV with headers
# features_df = pd.read_csv(features_path)


# features_df["Sequence_ID"] = features_df["Sequence_ID"].apply(lambda x: x.split('|')[1] if '|' in x else x)

# # Create feature dictionary (key: ID, value: 400-d vector)
# feature_dict = {
#     row["Sequence_ID"]: row.drop("Sequence_ID").values.astype(float)
#     for _, row in features_df.iterrows()
# }


# train_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\TrainSet1.csv"
# test_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\TestSet1.csv"

# train_df = pd.read_csv(train_csv_path)
# test_df = pd.read_csv(test_csv_path)


# def generate_formatted_lines(pairs_df, feature_dict):
#     lines = []
    
    
#     for _, row in pairs_df.iterrows():
#         id1 = str(row["IntA"])
#         id2 = str(row["IntB"])
#         label = str(row["Cls"])

#         if id1 in feature_dict and id2 in feature_dict:
#             vec1 = feature_dict[id1]
#             vec2 = feature_dict[id2]
#             combined = np.concatenate([vec1, vec2])
#             vec_str = ' '.join(map(str, combined))
#             line = f"{id1}+{id2}\t{vec_str}\t{label}"
#             lines.append(line)
#         else:
#             print(f"Skipping pair (missing vector): {id1}, {id2}")
            

    
#     return lines


# train_lines = generate_formatted_lines(train_df, feature_dict)
# test_lines = generate_formatted_lines(test_df, feature_dict)

# train_output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"
# test_output_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"

# with open(train_output_path, 'w') as f:
#     for line in train_lines:
#         f.write(line + '\n')

# with open(test_output_path, 'w') as f:
#     for line in test_lines:
#         f.write(line + '\n')

# # print(" Train and test datasets saved successfully!")



import pandas as pd
import numpy as np
import os

# --- Load feature vectors ---
features_path = r"C:\Users\cs21b\Desktop\Major_project_trail\ngram_feature_vector_new.csv"
features_df = pd.read_csv(features_path)
features_df["Sequence_ID"] = features_df["Sequence_ID"].apply(lambda x: x.split('|')[1] if '|' in x else x)

feature_dict = {
    row["Sequence_ID"]: row.drop("Sequence_ID").values.astype(float)
    for _, row in features_df.iterrows()
}

# --- Define dataset path structure ---
dataset_map = {
    "C1": [11, 12, 13, 14, 15],
    "C2": [21, 22, 23, 24, 25],
    "C3": [31, 32, 33, 34, 35]
}

base_input = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset"
base_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# --- Format generator ---
def generate_formatted_lines(pairs_df, feature_dict):
    lines = []
    for _, row in pairs_df.iterrows():
        id1 = str(row["IntA"])
        id2 = str(row["IntB"])
        # label = str(row["Cls"])
        if 'Cls' in row:
            label = row['Cls']
        elif 'Class' in row:
            label = row['Class']
        else:
            # print(" Label column not found in row")
            continue

        if id1 in feature_dict and id2 in feature_dict:
            vec1 = feature_dict[id1]
            vec2 = feature_dict[id2]
            combined = np.concatenate([vec1, vec2])
            vec_str = ' '.join(map(str, combined))
            line = f"{id1}+{id2}\t{vec_str}\t{label}"
            lines.append(line)
        else:
            print(f"Skipping pair (missing vector): {id1}, {id2}")
    return lines

# --- Process all datasets ---
for cat, nums in dataset_map.items():
    print(f"\n--- Processing Category {cat} ---")
    for num in nums:
        idx = str(num)[-1]

        train_path = fr"{base_input}\Dataset_{cat}\TrainSet{idx}.csv"
        test_path = fr"{base_input}\Dataset_{cat}\TestSet{idx}.csv"

        train_out = fr"{base_output}\For_{cat}\For_Train\train_dataset{num}.tsv"
        test_out = fr"{base_output}\For_{cat}\For_Test\test_dataset{num}.tsv"

        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)

        train_lines = generate_formatted_lines(train_df, feature_dict)
        test_lines = generate_formatted_lines(test_df, feature_dict)

        os.makedirs(os.path.dirname(train_out), exist_ok=True)
        os.makedirs(os.path.dirname(test_out), exist_ok=True)

        with open(train_out, 'w') as f:
            for line in train_lines:
                f.write(line + '\n')

        with open(test_out, 'w') as f:
            for line in test_lines:
                f.write(line + '\n')

        print(f"→ Saved: train_dataset{num}.tsv & test_dataset{num}.tsv\n")



--- Processing Category C1 ---
Skipping pair (missing vector): P0DI81, Q7Z392
Skipping pair (missing vector): P0DI81, Q8IUR0
Skipping pair (missing vector): O75376, Q9NNW7
Skipping pair (missing vector): P26678, O00631
Skipping pair (missing vector): O75865, P0DI81
Skipping pair (missing vector): Q92813, P62253
Skipping pair (missing vector): P55072, Q9BQE4
Skipping pair (missing vector): Q92813, O60337
Skipping pair (missing vector): Q96G74, P36969
Skipping pair (missing vector): Q9Y296, P0DI81
Skipping pair (missing vector): P63261, P62328
Skipping pair (missing vector): Q9BQE4, Q86TM6
Skipping pair (missing vector): P03372, Q16881
Skipping pair (missing vector): P67870, P62945
Skipping pair (missing vector): Q92813, Q9Y6I7
Skipping pair (missing vector): P06733, P0DI81
Skipping pair (missing vector): P68133, P62328
Skipping pair (missing vector): O43617, P0DI81
Skipping pair (missing vector): Q86SZ2, P0DI81
Skipping pair (missing vector): P40692, P62328
Skipping pair (missing vecto

In [9]:
# print("dhdf")



In [ ]:
# import pandas as pd
# import numpy as np

# # === Load the train .tsv file ===
# train_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"
# train_df = pd.read_csv(train_path, sep="\t", header=None)

# # Column structure: [ID1+ID2, vector_string, label]
# # Split the vector string into list of floats
# train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))

# # Extract X and Y
# Train_X = np.vstack(train_df[1].values)   # Shape: (num_samples, 800)
# Train_Y = train_df[2].astype(int).values  # Shape: (num_samples,)

# print("Train_X shape:", Train_X.shape)
# print("Train_Y shape:", Train_Y.shape)

# # === Same for test file ===
# test_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"
# test_df = pd.read_csv(test_path, sep="\t", header=None)

# test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
# Test_X = np.vstack(test_df[1].values)
# Test_Y = test_df[2].astype(int).values

# print("Test_X shape:", Test_X.shape)
# print("Test_Y shape:", Test_Y.shape)


In [ ]:
# import pandas as pd
# import numpy as np

# # === Load the train .tsv file ===
# train_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"
# train_df = pd.read_csv(train_path, sep="\t", header=None)

# # Column structure: [ID1+ID2, vector_string, label]
# # Split the vector string into list of floats
# train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))

# # Extract X and Y
# Train_X = np.vstack(train_df[1].values)              # Shape: (num_samples, 800)
# Train_Y = train_df[2].astype(int).values.reshape(-1, 1)  # Shape: (num_samples, 1)

# print("Train_X shape:", Train_X.shape)
# print("Train_Y shape:", Train_Y.shape)

# # === Same for test file ===
# test_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"
# test_df = pd.read_csv(test_path, sep="\t", header=None)

# test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
# Test_X = np.vstack(test_df[1].values)
# Test_Y = test_df[2].astype(int).values.reshape(-1, 1)

# print("Test_X shape:", Test_X.shape)
# print("Test_Y shape:", Test_Y.shape)

# # Preview: First 5 rows of Train_X and Train_Y




In [ ]:
import pandas as pd
import numpy as np


train_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"
train_df = pd.read_csv(train_path, sep="\t", header=None)


train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))


Train_X = np.vstack(train_df[1].values)              # Shape: (num_samples, 800)
Train_Y = train_df[2].astype(int).values.reshape(-1, 1)  # Shape: (num_samples, 1)

print(" Train_X shape:", Train_X.shape)
print(" Train_Y shape:", Train_Y.shape)


test_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"
test_df = pd.read_csv(test_path, sep="\t", header=None)

test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
Test_X = np.vstack(test_df[1].values)
Test_Y = test_df[2].astype(int).values.reshape(-1, 1)

print(" Test_X shape:", Test_X.shape)
print(" Test_Y shape:", Test_Y.shape)



✅ Train_X shape: (45380, 800)
✅ Train_Y shape: (45380, 1)
✅ Test_X shape: (5049, 800)
✅ Test_Y shape: (5049, 1)


In [ ]:
import pandas as pd
import numpy as np
import os

# --- Dataset map same as before ---
dataset_map = {
    "C1": [11, 12, 13, 14, 15],
    "C2": [21, 22, 23, 24, 25],
    "C3": [31, 32, 33, 34, 35]
}

base_output = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# --- Function to load tsv and extract features/labels ---
def load_dataset(tsv_path):
    df = pd.read_csv(tsv_path, sep="\t", header=None)
    df[1] = df[1].apply(lambda x: list(map(float, x.strip().split())))
    X = np.vstack(df[1].values)
    Y = df[2].astype(int).values.reshape(-1, 1)
    return X, Y

# --- Process all datasets ---
for cat, nums in dataset_map.items():
    print(f"\n Category: {cat}")
    for num in nums:
        train_path = fr"{base_output}\For_{cat}\For_Train\train_dataset{num}.tsv"
        test_path = fr"{base_output}\For_{cat}\For_Test\test_dataset{num}.tsv"

        if not os.path.exists(train_path) or not os.path.exists(test_path):
            print(f" Missing files for dataset {num}")
            continue

        Train_X, Train_Y = load_dataset(train_path)
        Test_X, Test_Y = load_dataset(test_path)

        print(f" Dataset {num}:")
        print(f"   🔹 Train_X shape: {Train_X.shape}")
        print(f"   🔹 Train_Y shape: {Train_Y.shape}")
        print(f"   🔹 Test_X shape : {Test_X.shape}")
        print(f"   🔹 Test_Y shape : {Test_Y.shape}")



📂 Category: C1
✅ Dataset 11:
   🔹 Train_X shape: (45380, 800)
   🔹 Train_Y shape: (45380, 1)
   🔹 Test_X shape : (5049, 800)
   🔹 Test_Y shape : (5049, 1)
✅ Dataset 12:
   🔹 Train_X shape: (45383, 800)
   🔹 Train_Y shape: (45383, 1)
   🔹 Test_X shape : (5046, 800)
   🔹 Test_Y shape : (5046, 1)
✅ Dataset 13:
   🔹 Train_X shape: (45380, 800)
   🔹 Train_Y shape: (45380, 1)
   🔹 Test_X shape : (5049, 800)
   🔹 Test_Y shape : (5049, 1)
✅ Dataset 14:
   🔹 Train_X shape: (45386, 800)
   🔹 Train_Y shape: (45386, 1)
   🔹 Test_X shape : (5043, 800)
   🔹 Test_Y shape : (5043, 1)
✅ Dataset 15:
   🔹 Train_X shape: (45384, 800)
   🔹 Train_Y shape: (45384, 1)
   🔹 Test_X shape : (5045, 800)
   🔹 Test_Y shape : (5045, 1)

📂 Category: C2
✅ Dataset 21:
   🔹 Train_X shape: (18940, 800)
   🔹 Train_Y shape: (18940, 1)
   🔹 Test_X shape : (1496, 800)
   🔹 Test_Y shape : (1496, 1)
✅ Dataset 22:
   🔹 Train_X shape: (18356, 800)
   🔹 Train_Y shape: (18356, 1)
   🔹 Test_X shape : (1497, 800)
   🔹 Test_Y shape 

In [ ]:
import pandas as pd
import numpy as np

# === Load the train .tsv file ===
train_path = r"C:\Users\cs21b\Desktop\Major_project_trail\train_dataset11.tsv"
train_df = pd.read_csv(train_path, sep="\t", header=None)

# Column structure: [ID1+ID2, vector_string, label]
# Split the vector string into list of floats
train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))

# Extract X and Y
Train_X = np.vstack(train_df[1].values)  # Shape: (num_samples, 800)
Train_Y = train_df[2].astype(int).values  # Shape: (num_samples, )

# Reshape Train_Y to be a single row (1, num_samples)
Train_Y = Train_Y.reshape(1, -1)  # Shape: (1, num_samples)

print("Train_X shape:", Train_X.shape)
print("Train_Y shape:", Train_Y.shape)

# === Same for test file ===
test_path = r"C:\Users\cs21b\Desktop\Major_project_trail\test_dataset11.tsv"
test_df = pd.read_csv(test_path, sep="\t", header=None)

test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
Test_X = np.vstack(test_df[1].values)
Test_Y = test_df[2].astype(int).values

# Reshape Test_Y to be a single row (1, num_samples)
Test_Y = Test_Y.reshape(1, -1)  # Shape: (1, num_samples)

print("Test_X shape:", Test_X.shape)
print("Test_Y shape:", Test_Y.shape)

# Preview: First 5 rows of Train_X and Train_Y
print("\n🔍 Train_Y preview:", Train_Y[:, :])  # Preview first 5 labels


✅ Train_X shape: (45380, 800)
✅ Train_Y shape: (1, 45380)
✅ Test_X shape: (5049, 800)
✅ Test_Y shape: (1, 5049)

🔍 Train_Y preview: [[1 1 1 ... 0 0 0]]


In [ ]:
import pandas as pd
import numpy as np
import os

# === Base path ===
base_path = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# === Dataset fold configurations ===
dataset_folds = [
    # C1
    (11, "C1"), (12, "C1"), (13, "C1"), (14, "C1"), (15, "C1"),
    # C2
    (21, "C2"), (22, "C2"), (23, "C2"), (24, "C2"), (25, "C2"),
    # C3
    (31, "C3"), (32, "C3"), (33, "C3"), (34, "C3"), (35, "C3"),
]

# === Iterate over each fold ===
for fold_num, category in dataset_folds:
    print(f"\n DataSet Folder:= {category}: set-{fold_num}")

    train_path = os.path.join(base_path, f"For_{category}", "For_Train", f"train_dataset{fold_num}.tsv")
    test_path = os.path.join(base_path, f"For_{category}", "For_Test", f"test_dataset{fold_num}.tsv")

    # === Load Train Set ===
    train_df = pd.read_csv(train_path, sep="\t", header=None)
    train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Train_X = np.vstack(train_df[1].values)
    Train_Y = train_df[2].astype(int).values.reshape(1, -1)

    print(f" Train_X shape: {Train_X.shape}")
    print(f" Train_Y shape: {Train_Y.shape}")

    # === Load Test Set ===
    test_df = pd.read_csv(test_path, sep="\t", header=None)
    test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Test_X = np.vstack(test_df[1].values)
    Test_Y = test_df[2].astype(int).values.reshape(1, -1)

    print(f" Test_X shape: {Test_X.shape}")
    print(f" Test_Y shape: {Test_Y.shape}")

    # # === Optional Preview ===
    # print(f" First 5 Train_Y: {Train_Y[:, :5]}")



 Processing Fold: C1 - 11
 Train_X shape: (45380, 800)
 Train_Y shape: (1, 45380)
 Test_X shape: (5049, 800)
 Test_Y shape: (1, 5049)

 Processing Fold: C1 - 12
 Train_X shape: (45383, 800)
 Train_Y shape: (1, 45383)
 Test_X shape: (5046, 800)
 Test_Y shape: (1, 5046)

 Processing Fold: C1 - 13
 Train_X shape: (45380, 800)
 Train_Y shape: (1, 45380)
 Test_X shape: (5049, 800)
 Test_Y shape: (1, 5049)

 Processing Fold: C1 - 14
 Train_X shape: (45386, 800)
 Train_Y shape: (1, 45386)
 Test_X shape: (5043, 800)
 Test_Y shape: (1, 5043)

 Processing Fold: C1 - 15
 Train_X shape: (45384, 800)
 Train_Y shape: (1, 45384)
 Test_X shape: (5045, 800)
 Test_Y shape: (1, 5045)

 Processing Fold: C2 - 21
 Train_X shape: (18940, 800)
 Train_Y shape: (1, 18940)
 Test_X shape: (1496, 800)
 Test_Y shape: (1, 1496)

 Processing Fold: C2 - 22
 Train_X shape: (18356, 800)
 Train_Y shape: (1, 18356)
 Test_X shape: (1497, 800)
 Test_Y shape: (1, 1497)

 Processing Fold: C2 - 23
 Train_X shape: (18982, 800)

In [17]:
import pandas as pd
import numpy as np
import os

# === Base path ===
base_path = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# === Dataset fold configurations ===
dataset_folds = [
    # C1
    (11, "C1"), (12, "C1"), (13, "C1"), (14, "C1"), (15, "C1"),
    # C2
    (21, "C2"), (22, "C2"), (23, "C2"), (24, "C2"), (25, "C2"),
    # C3
    (31, "C3"), (32, "C3"), (33, "C3"), (34, "C3"), (35, "C3"),
]

# === Store all test data ===
all_test_data = {}

# === Iterate over each fold ===
for fold_num, category in dataset_folds:
    print(f"\n DataSet Folder:= {category}: set-{fold_num}")

    train_path = os.path.join(base_path, f"For_{category}", "For_Train", f"train_dataset{fold_num}.tsv")
    test_path = os.path.join(base_path, f"For_{category}", "For_Test", f"test_dataset{fold_num}.tsv")

    # === Load Train Set ===
    train_df = pd.read_csv(train_path, sep="\t", header=None)
    train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Train_X = np.vstack(train_df[1].values)
    Train_Y = train_df[2].astype(int).values.reshape(1, -1)

    print(f" Train_X shape: {Train_X.shape}")
    print(f" Train_Y shape: {Train_Y.shape}")

    # === Load Test Set ===
    test_df = pd.read_csv(test_path, sep="\t", header=None)
    test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Test_X = np.vstack(test_df[1].values)
    Test_Y = test_df[2].astype(int).values.reshape(1, -1)

    print(f" Test_X shape: {Test_X.shape}")
    print(f" Test_Y shape: {Test_Y.shape}")

    # === Store Test Data ===
    if category not in all_test_data:
        all_test_data[category] = {}

    all_test_data[category][fold_num] = {
        "Test_X": Test_X,
        "Test_Y": Test_Y
    }



 DataSet Folder:= C1: set-11
 Train_X shape: (45380, 800)
 Train_Y shape: (1, 45380)
 Test_X shape: (5049, 800)
 Test_Y shape: (1, 5049)

 DataSet Folder:= C1: set-12
 Train_X shape: (45383, 800)
 Train_Y shape: (1, 45383)
 Test_X shape: (5046, 800)
 Test_Y shape: (1, 5046)

 DataSet Folder:= C1: set-13
 Train_X shape: (45380, 800)
 Train_Y shape: (1, 45380)
 Test_X shape: (5049, 800)
 Test_Y shape: (1, 5049)

 DataSet Folder:= C1: set-14
 Train_X shape: (45386, 800)
 Train_Y shape: (1, 45386)
 Test_X shape: (5043, 800)
 Test_Y shape: (1, 5043)

 DataSet Folder:= C1: set-15
 Train_X shape: (45384, 800)
 Train_Y shape: (1, 45384)
 Test_X shape: (5045, 800)
 Test_Y shape: (1, 5045)

 DataSet Folder:= C2: set-21
 Train_X shape: (18940, 800)
 Train_Y shape: (1, 18940)
 Test_X shape: (1496, 800)
 Test_Y shape: (1, 1496)

 DataSet Folder:= C2: set-22
 Train_X shape: (18356, 800)
 Train_Y shape: (1, 18356)
 Test_X shape: (1497, 800)
 Test_Y shape: (1, 1497)

 DataSet Folder:= C2: set-23
 Tra

In [13]:

%pip install scikit-learn



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\cs21b\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
# import pandas as pd
# import numpy as np
# import os
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# # === Base path ===
# base_path = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# # === Dataset fold configurations ===
# dataset_folds = [
#     # C1
#     (11, "C1"), (12, "C1"), (13, "C1"), (14, "C1"), (15, "C1"),
#     # C2
#     (21, "C2"), (22, "C2"), (23, "C2"), (24, "C2"), (25, "C2"),
#     # C3
#     (31, "C3"), (32, "C3"), (33, "C3"), (34, "C3"), (35, "C3"),
# ]

# # === Classifiers ===
# classifiers = {
#     "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
#     # "SVM (RBF Kernel)": SVC(kernel='rbf', random_state=42),
#     "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
# }

# # === To store all test data ===
# all_test_data = {}
# all_results = []

# # === Iterate over each fold ===
# for fold_num, category in dataset_folds:
#     print(f"\n DataSet Folder: {category} | Set-{fold_num}")

#     train_path = os.path.join(base_path, f"For_{category}", "For_Train", f"train_dataset{fold_num}.tsv")
#     test_path = os.path.join(base_path, f"For_{category}", "For_Test", f"test_dataset{fold_num}.tsv")

#     # === Load Train Set ===
#     train_df = pd.read_csv(train_path, sep="\t", header=None)
#     train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))
#     Train_X = np.vstack(train_df[1].values)
#     Train_Y = train_df[2].astype(int).values.ravel()

#     # === Load Test Set ===
#     test_df = pd.read_csv(test_path, sep="\t", header=None)
#     test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
#     Test_X = np.vstack(test_df[1].values)
#     Test_Y = test_df[2].astype(int).values.ravel()

#     # === Store test data for future use ===
#     if category not in all_test_data:
#         all_test_data[category] = {}
#     all_test_data[category][fold_num] = {
#         "Test_X": Test_X,
#         "Test_Y": Test_Y
#     }

#     # === Train and evaluate classifiers ===
#     for name, clf in classifiers.items():
#         print(f" Training: {name} on {category} set-{fold_num}")
#         clf.fit(Train_X, Train_Y)

#         train_pred = clf.predict(Train_X)
#         test_pred = clf.predict(Test_X)

#         train_acc = accuracy_score(Train_Y, train_pred)
#         test_acc = accuracy_score(Test_Y, test_pred)

#         all_results.append({
#             "Category": category,
#             "Fold": fold_num,
#             "Classifier": name,
#             "Train_Accuracy": round(train_acc * 100, 2),
#             "Test_Accuracy": round(test_acc * 100, 2)
#         })

# # === Print Summary ===
# print("\n Accuracy Summary")
# print("{:<10} {:<6} {:<25} {:<15} {:<15}".format("Category", "Fold", "Classifier", "Train Accuracy", "Test Accuracy"))
# print("-" * 75)
# for res in all_results:
#     print("{:<10} {:<6} {:<25} {:<15.2f} {:<15.2f}".format(
#         res["Category"], res["Fold"], res["Classifier"], res["Train_Accuracy"], res["Test_Accuracy"]
#     ))


import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, average_precision_score,
    matthews_corrcoef
)

# === Update base path accordingly ===
base_path = r"C:\Users\cs21b\Desktop\Major_project_trail\Dataset_output"

# === Dataset fold configurations ===
dataset_folds = [
    (11, "C1"), (12, "C1"), (13, "C1"), (14, "C1"), (15, "C1"),
    (21, "C2"), (22, "C2"), (23, "C2"), (24, "C2"), (25, "C2"),
    (31, "C3"), (32, "C3"), (33, "C3"), (34, "C3"), (35, "C3"),
]


# === Classifiers ===
classifiers = {
    # "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM (RBF Kernel)": SVC(kernel='rbf', probability=True, random_state=42),
    # "SVM (RBF Kernel)": SVC(kernel='rbf', random_state=42),
    # "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}

# === Evaluation & Result Storage ===
final_results = []

for fold_num, category in dataset_folds:
    
    # === Load training set (static base training set) ===
    train_path = os.path.join(base_path, f"For_{category}", "For_Train", f"train_dataset{fold_num}.tsv")
    train_df = pd.read_csv(train_path, sep="\t", header=None)
    train_df[1] = train_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Train_X = np.vstack(train_df[1].values)
    Train_Y = train_df[2].astype(int).values.ravel()




    test_path = os.path.join(base_path, f"For_{category}", "For_Test", f"test_dataset{fold_num}.tsv")
    test_df = pd.read_csv(test_path, sep="\t", header=None)
    test_df[1] = test_df[1].apply(lambda x: list(map(float, x.strip().split())))
    Test_X = np.vstack(test_df[1].values)
    Test_Y = test_df[2].astype(int).values.ravel()
    print(fold_num, category,Train_X.shape, Test_X.shape)
    for name, clf in classifiers.items():
        clf.fit(Train_X, Train_Y)
        test_pred = clf.predict(Test_X)

        try:
            test_proba = clf.predict_proba(Test_X)[:, 1]
        except:
            test_proba = None

        acc = accuracy_score(Test_Y, test_pred)
        f1 = f1_score(Test_Y, test_pred)
        mcc = matthews_corrcoef(Test_Y, test_pred)
        auc = roc_auc_score(Test_Y, test_proba) if test_proba is not None else None
        auprc = average_precision_score(Test_Y, test_proba) if test_proba is not None else None

        final_results.append([
            category, fold_num, name,
            round(acc * 100, 2), round(f1, 4), round(mcc, 4),
            round(auc, 4) if auc is not None else "NA",
            round(auprc, 4) if auprc is not None else "NA"
        ])


df_final = pd.DataFrame(final_results, columns=[
    "Category", "Fold", "Classifier",
    "Accuracy", "F1 Score", "MCC", "AUC", "AUPRC"
])

output_csv_path = r"C:\Users\cs21b\Desktop\Major_project_trail\combined_results_summary_new_svm.csv"
df_final.to_csv(output_csv_path, index=False)
print(f"\n All evaluation completed and results saved to: {output_csv_path}")


11 C1 (45380, 800) (5049, 800)
12 C1 (45383, 800) (5046, 800)
13 C1 (45380, 800) (5049, 800)
14 C1 (45386, 800) (5043, 800)
15 C1 (45384, 800) (5045, 800)
21 C2 (18940, 800) (1496, 800)
22 C2 (18356, 800) (1497, 800)
23 C2 (18982, 800) (1488, 800)
24 C2 (19237, 800) (1491, 800)
25 C2 (19176, 800) (1499, 800)
31 C3 (21512, 800) (5923, 800)
32 C3 (21964, 800) (5777, 800)
33 C3 (20898, 800) (6264, 800)
34 C3 (20980, 800) (6363, 800)
35 C3 (21472, 800) (6153, 800)

 All evaluation completed and results saved to: C:\Users\cs21b\Desktop\Major_project_trail\combined_results_summary_new_svm.csv


In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score


# rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# rf_clf.fit(Train_X, Train_Y.ravel()) 

# Train_Y_pred = rf_clf.predict(Train_X)

# train_accuracy = accuracy_score(Train_Y, Train_Y_pred)
# print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Test_Y_pred = rf_clf.predict(Test_X)
# test_accuracy = accuracy_score(Test_Y, Test_Y_pred)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")




            Classifier  Train Accuracy  Test Accuracy
0        Random Forest      100.000000      79.302977
1  K-Nearest Neighbors       97.723667      84.913663


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Flatten labels (in case they are shaped like (1, N))
Train_Y = Train_Y.ravel()
Test_Y = Test_Y.ravel()

# Dictionary of classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM (RBF Kernel)": SVC(kernel='rbf', random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}

results = []

# Loop through classifiers
for name, clf in classifiers.items():
    print(f"\n Training: {name}")
    clf.fit(Train_X, Train_Y)

    # Predictions
    train_pred = clf.predict(Train_X)
    test_pred = clf.predict(Test_X)

    # Accuracy
    train_acc = accuracy_score(Train_Y, train_pred)
    test_acc = accuracy_score(Test_Y, test_pred)

    results.append((name, train_acc * 100, test_acc * 100))

print("\n Accuracy Summary:")
print("{:<25} {:<15} {:<15}".format("Classifier", "Train Accuracy", "Test Accuracy"))
print("-" * 55)
for name, train_acc, test_acc in results:
    print("{:<25} {:<15.2f} {:<15.2f}".format(name, train_acc, test_acc))



🔷 Training: Random Forest

🔷 Training: SVM (RBF Kernel)

🔷 Training: K-Nearest Neighbors

 Accuracy Summary:
Classifier                Train Accuracy  Test Accuracy  
-------------------------------------------------------
Random Forest             100.00          59.30          
SVM (RBF Kernel)          98.66           83.98          
K-Nearest Neighbors       97.72           86.79          


In [18]:
print(Train_X.shape)

print(Train_Y.shape)

(45380, 800)
(45380,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, average_precision_score,
    matthews_corrcoef, classification_report, confusion_matrix
)

# === Evaluation Function ===
def evaluate_model(name, y_true, y_pred, y_proba=None):
    print(f"\n Evaluation Metrics for: {name}")
    
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)

    if y_proba is not None:
        auc = roc_auc_score(y_true, y_proba)
        auprc = average_precision_score(y_true, y_proba)
    else:
        auc = auprc = None

    print(f" Accuracy       : {acc * 100:.2f}%")
    print(f" F1 Score       : {f1:.4f}")
    print(f" ROC-AUC        : {auc:.4f}" if auc is not None else "📈 ROC-AUC        : Not Available")
    print(f" PR-AUC (AUPRC) : {auprc:.4f}" if auprc is not None else "🔁 PR-AUC (AUPRC) : Not Available")
    print(f" MCC            : {mcc:.4f}")
    
    print("\n Classification Report:")
    print(classification_report(y_true, y_pred))

    print(" Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("-" * 60)

# === Flatten labels ===
Train_Y = Train_Y.ravel()
Test_Y = Test_Y.ravel()

# === Classifier Dictionary ===
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM (RBF Kernel)": SVC(kernel='rbf', probability=True, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5)
}

results = []

# === Loop & Evaluate ===
for name, clf in classifiers.items():
    print(f"\n Training: {name}")
    clf.fit(Train_X, Train_Y)

    # train_pred = clf.predict(Train_X)
    test_pred = clf.predict(Test_X)

    # For AUC, AUPRC → need probability of class 1
    try:
        test_proba = clf.predict_proba(Test_X)[:, 1]
    except:
        test_proba = None  # Not all classifiers support predict_proba

    # Store accuracy for summary
    # train_acc = accuracy_score(Train_Y, train_pred)
    test_acc = accuracy_score(Test_Y, test_pred)
    results.append((name, test_acc * 100))

    # Evaluate and print full metrics
    evaluate_model(name, Test_Y, test_pred, test_proba)

# === Accuracy Summary ===
print("\n Accuracy Summary Table:")
print("{:<25} {:<15} {:<15}".format("Classifier", "Train Accuracy", "Test Accuracy"))
print("-" * 55)
for name, train_acc, test_acc in results:
    print("{:<25} {:<15.2f} {:<15.2f}".format(name, train_acc, test_acc))



🔷 Training: Random Forest

📌 Evaluation Metrics for: Random Forest
✅ Accuracy       : 59.30%
🎯 F1 Score       : 0.7094
📈 ROC-AUC        : 0.9736
🔁 PR-AUC (AUPRC) : 0.9809
🧠 MCC            : 0.3102

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.19      0.32      2524
           1       0.55      0.99      0.71      2525

    accuracy                           0.59      5049
   macro avg       0.76      0.59      0.52      5049
weighted avg       0.76      0.59      0.52      5049

📌 Confusion Matrix:
[[ 486 2038]
 [  17 2508]]
------------------------------------------------------------

🔷 Training: SVM (RBF Kernel)

📌 Evaluation Metrics for: SVM (RBF Kernel)
✅ Accuracy       : 83.98%
🎯 F1 Score       : 0.8580
📈 ROC-AUC        : 0.9109
🔁 PR-AUC (AUPRC) : 0.8660
🧠 MCC            : 0.7030

📊 Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.71      0.82      2524
  

In [1]:
help(RandomForestClassifier)
help(SVC)
help(KNeighborsClassifier)



NameError: name 'RandomForestClassifier' is not defined